In [1]:
# Import Libraries
import pickle
import json
import pandas as pd
import numpy as np

In [2]:
# Load Model

with open('windsoriser_extreme.pkl', 'rb') as file_1:
  windsoriser_extreme = pickle.load(file_1)

with open('windsoriser_skew.pkl', 'rb') as file_2:
  windsoriser_skew = pickle.load(file_2)

with open('scaler.pkl', 'rb') as file_3:
  scaler = pickle.load(file_3)

with open('pca_final.pkl', 'rb') as file_4:
  pca_final = pickle.load(file_4)

with open('km_final.pkl', 'rb') as file_5:
  km_final = pickle.load(file_5)

with open('extreme_skew.txt', 'r') as file_6:
  extreme_skew = json.load(file_6)

with open('not_extreme_skew.txt', 'r') as file_7:
  not_extreme_skew = json.load(file_7)

with open('normal.txt', 'r') as file_8:
  normal = json.load(file_8)


In [3]:
# Create New Data 

data_inf = {
    'BALANCE' : 2047,
 'BALANCE_FREQUENCY' : 1,
 'PURCHASES' : 1102,
 'ONEOFF_PURCHASES' : 575,
 'INSTALLMENTS_PURCHASES' :455,
 'CASH_ADVANCE' :1138,
 'PURCHASES_FREQUENCY' : 0.9,
 'ONEOFF_PURCHASES_FREQUENCY' : 0.25,
 'PURCHASES_INSTALLMENTS_FREQUENCY' :0.75,
 'CASH_ADVANCE_FREQUENCY' :0.25,
 'CASH_ADVANCE_TRX' :4,
 'PURCHASES_TRX' :17,
 'CREDIT_LIMIT' :6500,
 'PAYMENTS' :1911,
 'MINIMUM_PAYMENTS' : 785,
 'PRC_FULL_PAYMENT' :0.12,
 'TENURE' : 12
}

data_inf = pd.DataFrame([data_inf])
data_inf

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,2047,1,1102,575,455,1138,0.9,0.25,0.75,0.25,4,17,6500,1911,785,0.12,12


In [4]:
#Membagi DF sesuai dengan skew-nya
df_extreme_skew = data_inf[extreme_skew]
df_skew = data_inf[not_extreme_skew]
df_normal = data_inf[normal]

In [5]:
#Transform ke windsorizer
df_wins_extreme = windsoriser_extreme.transform(df_extreme_skew)
df_wins_skew = windsoriser_skew.transform(df_skew)

In [6]:
# Concat 3 dataframe
df_censor = pd.concat([df_wins_extreme,df_wins_skew,df_normal,data_inf['TENURE']], axis=1)
df_censor = df_censor.loc[:, ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']]
df_censor

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,2047,1,1102,575,455,1138,0.9,0.25,0.75,0.25,4,17,6500,1911,785,0.12,12


In [7]:
#Scaling pada setiap feature
df_scaled = pd.DataFrame(scaler.transform(df_censor), columns=df_censor.columns.to_list())
df_scaled

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,0.611316,0.0,0.698116,0.920257,0.803668,0.999221,0.48,0.666668,0.777777,1.0,1.0,0.625,0.714286,0.673432,0.765269,0.96,0.0


In [8]:
#Reduksi komponen menggunakan PCA
X_pca = pca_final.transform(df_scaled)
X_pca

array([[ 0.91266351,  0.77482995, -0.12635344, -0.59038552, -0.09121023,
        -0.13730569, -0.07573276]])

In [9]:
# Predict using Random Forest Opt
y_pred_inf = km_final.predict(X_pca)
y_pred_inf

array([0])